## Notebook 2.1 Data Understanding and Preprocessing of Support Tables

For all intents and purposes, this should be considered as the second real notebook that is part of the thesis work. In it, we will look at the support tables that are part of the NOVA IMS original database.

#### 1. We are familiarized with the general structure of the logs

Before going further, we should assess the remaining tables presented in the database. 

Recall, **logs record interactions with the system and we are looking for ways to determine whether these interactions can assist educators identify at risk students and high performing students.**

Thus, to make the best out of the logs, we will need to perform different segmentations and it is likely that we will need perform some filtering. 

### To do that, we will take a look at all tables

We will look at all tables and all columns to make a preliminary assessment of the utility of the available elements.
In general, these are support elements that will be used sparsely, as most of the relevant information is present in the logs.

The observation of each table will resort to the same chain of commands:

info -> to observe count and datatype of each column, 
describe -> a command that that returns the most notable descriptive statistics of each column.
The obeservation of each table ends with a look at the raw data (At least the visible rows).

#### 2. We'll start this notebook by importing all relevant packages and data

All data is stored in an excel file.

In [1]:
#import libs
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm, trange
tqdm.pandas(desc="Progress")

sns.set()

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
#other tables with support information
support_table = pd.read_excel('../Data/Nova_IMS_logs_Moodle_cursos.xlsx', sheet_name = None,
                             dtype = {
                                 'cd_lectivo' : object,
                                 'cd_curso' : object,
                                 'cd_Discip': object,
                                 'cd_discip': object,
                                 'userId': object,
                                 'dataExame': pd.datetime,
                             })

support_table['logs_Moodle_cursos'].rename(columns = {
                                 'cd_Discip' : 'courseid',
                                 'userId' : 'userid',
                             }, inplace = True)

In [3]:
#use this cell to write any additional piece of code that may be required

We can see that the support table dict is composed by 2 distinct tables.

Student performance is, in general, measured by the student's grade. So... how do we measure grades?
In our data, we have access to different grades - Exam/assignment and Final Grades.

In [4]:
support_table['logs_Moodle_cursos'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56651 entries, 0 to 56650
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cd_lectivo       56651 non-null  object 
 1   cd_curso         56651 non-null  object 
 2   nm_curso_pt      56651 non-null  object 
 3   nm_ramo          56651 non-null  object 
 4   courseid         56651 non-null  object 
 5   ds_discip_pt     56651 non-null  object 
 6   semestre         56651 non-null  object 
 7   userid           56651 non-null  object 
 8   statusAvaliacao  56651 non-null  object 
 9   statusEpoca      56651 non-null  object 
 10  notaAvaliacao    56651 non-null  float64
 11  cd_final         56651 non-null  object 
 12  statusFinal      56651 non-null  object 
 13  notaFinal        55486 non-null  float64
dtypes: float64(2), object(12)
memory usage: 6.1+ MB


In [5]:
support_table['logs_Moodle_cursos'].describe(include = 'all', datetime_is_numeric = True).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
cd_lectivo,56651.0,1.0,202021.0,56651.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cd_curso,56651.0,28.0,9434.0,11753.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nm_curso_pt,56651,28,Mestrado em Gestão de Informação,11753,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nm_ramo,56651,28,Ramo Comum,21742,NaN,NaN,NaN,NaN,NaN,NaN,NaN
courseid,56651.0,251.0,200071.0,1749.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ds_discip_pt,56651,239,Gestão do Conhecimento,1749,NaN,NaN,NaN,NaN,NaN,NaN,NaN
semestre,56651,6,S1,24337,NaN,NaN,NaN,NaN,NaN,NaN,NaN
userid,56651.0,2359.0,3141.0,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
statusAvaliacao,56651,28,Normal,15021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
statusEpoca,56651,8,Avaliado,36828,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can start by removing identifiers of courses and programs. These will will provide no additional information for our analysis. Additionally, as all of the logs refer to the same school year, the reference to them will promptly be removed.

In [6]:
#cd Lectivo is a single value column ~- other meaningless columns may also be
support_table['logs_Moodle_cursos'].drop(['cd_lectivo', 'nm_ramo'], axis = 1, inplace = True)

Before going forward, let us take a closer look at some columns - specifically the unique values each of these columns may take:
1. Semestre: Will provide valuable insights into how to split the data, 
2. statusAvaliação: Teels us to which phase a grade refers to,
3. statusEpoca: Refers to the grading status of a particular item,
4. statusFinal

In [7]:
#store columns of interest in list
columns_of_interest = ['semestre', 'statusAvaliacao', 'statusEpoca', 'statusFinal']

#print value counts of each of them:
for i in tqdm(columns_of_interest):
    print(f'Unique Values of Column {i}: \n')
    print(support_table['logs_Moodle_cursos'][i].value_counts())
    print('\n')

  0%|          | 0/4 [00:00<?, ?it/s]

Unique Values of Column semestre: 

S1    24337
S2    22404
T2     2855
T1     2484
T4     2442
T3     2129
Name: semestre, dtype: int64


Unique Values of Column statusAvaliacao: 

Normal                         15021
Nota Parcial 1                 12406
Nota Parcial 2                 10635
Nota Parcial 3                  7663
Nota Parcial 4                  4148
Nota Parcial 5                  1945
Recurso                         1341
Melhoria (2ª época)             1057
Nota Parcial 6                   784
Nota Parcial 7                   666
Nota Parcial 8                   207
Exame Época Final Normal         160
Nota Parcial 11                  146
Nota Parcial 9                   114
Época Especial                   111
Creditação                        81
Nota Parcial 10                   65
Nota Parcial 13                   49
Melhoria - Nota Parcial 1         12
Melhoria (1ª Época)               11
Melhoria - Nota Parcial 2          9
Melhoria - Nota Parcial 3          6
Esta

We now know a lot of different things.

**Next, we have the datasExames table**

This table stores important information concerning the different curricular units and their exam dates. 

In [8]:
support_table['datasExames'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3056 entries, 0 to 3055
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   cd_lectivo      3056 non-null   object        
 1   semestre        3056 non-null   object        
 2   cd_discip       3056 non-null   object        
 3   epocaAvaliacao  3056 non-null   object        
 4   dataExame       3056 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 119.5+ KB


In [9]:
support_table['datasExames'].describe(include = 'all', datetime_is_numeric = True).T

,count,unique,top,freq,mean,min,25%,50%,75%,max
cd_lectivo,3056.0,1.0,202021.0,3056.0,NaT,NaT,NaT,NaT,NaT,NaT
semestre,3056,6,S1,1470,NaT,NaT,NaT,NaT,NaT,NaT
cd_discip,3056.0,214.0,200188.0,84.0,NaT,NaT,NaT,NaT,NaT,NaT
epocaAvaliacao,3056,9,Normal,841,NaT,NaT,NaT,NaT,NaT,NaT
dataExame,3056,NaN,NaN,NaN,2021-04-14 19:08:19.476439808,2020-10-26,2021-01-26,2021-04-05,2021-06-30,2021-12-18


In [10]:
#cd Lectivo is a single value column - that is likely to refer to propbably referes
support_table['datasExames'] = support_table['datasExames'].drop(['cd_lectivo'], axis = 1).rename(columns = {'cd_discip': 'courseid'})
support_table['datasExames']

,semestre,courseid,epocaAvaliacao,dataExame
0,S1,200219,Normal,2020-12-18
1,S1,200219,Recurso,2021-02-03
2,S1,200219,Melhoria (2ª época),2021-02-03
3,S1,200060,Normal,2020-12-18
4,S1,200060,Recurso,2021-02-01
...,...,...,...,...
3051,S2,910016,Normal,2021-07-23
3052,S2,910017,Normal,2021-07-21
3053,S2,200247,Normal,2021-07-19
3054,S2,71119,Normal,2021-07-19


#### Before going forward, we have to consider the following:

In this instance, all courses have grades. Be it exam grade or final grade. As a first indicator, we will not consider grade improvements. 

These results come from students that have effectively completed the curricular unit. Likewise, we will not consider grades of special season - Reason being that these exams are only accessible to students that fulfill very strict conditions.

In this instance, it does not make sense to distinguish between mandatory and optional assignments. We will keep the different assignments listed in the hope we can associate an assignment to a specific time-stamp.

Additionally, we can already start to address course duration:
1. We need a start date and an end date.
2. Moodle Logs usually have a start date for the course: It is unclear, at this moment, whether the date presented therein is reflective of the actual start date or the course registry date. Regardless, it may possible for us to use the semester denomination to make a reasonable inference for duration using the weeks of start and finish.
3. The end date is given by the Normal exam data

In [11]:
#we create a list of items to remove from the logs
invalid_keys = ['Melhoria - Nota Parcial 1', #partial for improv
                'Melhoria - Nota Parcial 2', #partial 2
                'Melhoria - Nota Parcial 3', #partial 3          
                'Melhoria - Nota Parcial 4', #...
                'Melhoria - Nota Parcial 5',          
                'Melhoria - Nota Parcial 6',          
                'Época Especial', #special season
                'Estatuto especial - Exame 1', #extraordinarily special season exam        
                'Estatuto Especial - Exame 2', #extraordinarily special season exam 2 
                'Melhoria (1ª Época)', #improv season 1
                'Melhoria (2ª época)', #improv season 2
                'Creditação', #credits with grade season
               ]

#we remove the entries from grades
support_table['logs_Moodle_cursos'] = support_table['logs_Moodle_cursos'][~(support_table['logs_Moodle_cursos']['statusAvaliacao'].isin(invalid_keys))].reset_index(drop = True)

#for course duration, we will only care about the date of the first season - as it is more indicative of duration than other options
normal_season = ['Exame Época Final Normal', 'Normal']

#and only keep normal season dates
support_table['datasExames'] = support_table['datasExames'][support_table['datasExames']['epocaAvaliacao'].isin(normal_season)].reset_index(drop = True)

#first, make distinction between passes and fails 
support_table['datasExames']['epocaAvaliacao'] = np.where(support_table['datasExames']['epocaAvaliacao'] == 'Exame Época Final Normal',
                                                             'Normal', #getting rid of multiclassification
                                                             support_table['datasExames']['epocaAvaliacao'])

support_table['datasExames'].sort_values(by = ['semestre', 'courseid', 'dataExame']).drop_duplicates().reset_index(inplace = True)

**Our main target will be the exam grade** -> reason being that it is not directly computed from the grades of the different assignments. Whereas finalgrade results from these.

In [12]:
#first, make distinction between passes and fails 
support_table['logs_Moodle_cursos']['statusEpoca'] = np.where(support_table['logs_Moodle_cursos']['statusEpoca'] != 'Aprovado',
                                                             'Reprovado', #getting rid of multiclassification
                                                             support_table['logs_Moodle_cursos']['statusEpoca'])

#first, make distinction between passes and fails 
support_table['logs_Moodle_cursos']['statusAvaliacao'] = np.where(support_table['logs_Moodle_cursos']['statusAvaliacao'] == 'Exame Época Final Normal',
                                                             'Normal', #getting rid of multiclassification
                                                             support_table['logs_Moodle_cursos']['statusAvaliacao'])

support_table['logs_Moodle_cursos'] = support_table['logs_Moodle_cursos'].sort_values(by = ['semestre', 'courseid', 'userid', 'statusAvaliacao', 'notaAvaliacao'])
support_table['logs_Moodle_cursos'].drop_duplicates(subset = ['semestre', 'courseid', 'userid', 'statusAvaliacao'], inplace = True)

support_table['logs_Moodle_cursos']

,cd_curso,nm_curso_pt,courseid,ds_discip_pt,semestre,userid,statusAvaliacao,statusEpoca,notaAvaliacao,cd_final,statusFinal,notaFinal
10175,8259,Licenciatura em Sistemas e Tecnologias de Info...,100001,Álgebra Linear,S1,1279,Nota Parcial 1,Reprovado,8.6,N,Reprovado,NaN
12440,8259,Licenciatura em Sistemas e Tecnologias de Info...,100001,Álgebra Linear,S1,1544,Recurso,Reprovado,4.0,S,Reprovado,4.0
11890,8259,Licenciatura em Sistemas e Tecnologias de Info...,100001,Álgebra Linear,S1,1556,Normal,Reprovado,9.0,N,Reprovado,6.0
11891,8259,Licenciatura em Sistemas e Tecnologias de Info...,100001,Álgebra Linear,S1,1556,Nota Parcial 1,Reprovado,13.9,N,Reprovado,6.0
11892,8259,Licenciatura em Sistemas e Tecnologias de Info...,100001,Álgebra Linear,S1,1556,Nota Parcial 2,Reprovado,8.7,N,Reprovado,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
55317,4969,Pós-Graduação em Digital Enterprise Management,400094,Big Data Analytics for Business,T4,7250,Normal,Aprovado,18.0,S,Aprovado,18.0
55318,4969,Pós-Graduação em Digital Enterprise Management,400094,Big Data Analytics for Business,T4,7250,Nota Parcial 2,Reprovado,17.0,N,Aprovado,18.0
55319,4969,Pós-Graduação em Digital Enterprise Management,400094,Big Data Analytics for Business,T4,7250,Nota Parcial 3,Reprovado,18.0,N,Aprovado,18.0
55320,4969,Pós-Graduação em Digital Enterprise Management,400094,Big Data Analytics for Business,T4,7250,Nota Parcial 4,Reprovado,17.0,N,Aprovado,18.0


In [13]:
exames = ['Normal', 'Recurso']

#first, split exams from remaining rows
exams = support_table['logs_Moodle_cursos'][support_table['logs_Moodle_cursos']['statusAvaliacao'].isin(exames)].filter(['cd_curso', 'nm_curso_pt', 'courseid', 
                                                                                                                         'ds_discip_pt', 'semestre','userid', 
                                                                                                            'statusAvaliacao', 'notaAvaliacao',
                                                                                                           'notaFinal'])

#then, create a pivot table with the normal + recurso season exam grades 
exam_pivot = pd.pivot(exams, index = ['cd_curso', 'courseid', 'semestre','userid'], columns = 'statusAvaliacao', 
                 values = 'notaAvaliacao')

exam_pivot.dropna(how = 'all', inplace = True)

# We look at the exams at Normal and Recurso Season

We have 1341 student /course pairs that have take the 2nd season exam. 
We also have 329 student who have not taken the 1st season exam.

We can take a simple approach - fill all nans in Normal exam with the Recurso grade.

For that, we will check whether both dfs are the same.

In [14]:
#This cell is a relic from a previous formulation of the problem

# #we need to look for instances of nans in normal and not nas in recurso
# normal_nans = exam_pivot[exam_pivot['Normal'].isna()]
# valid_recurso = exam_pivot[exam_pivot['Recurso'].notna()]

# print(f'Are both dataframes exactly the same? \n' +
#       f'Answer: {normal_nans.equals(valid_recurso)}.')

# #del normal_nans, valid_recurso

**In light of the previous result**, we will look to join both columns together and get a valid targets_table - with exam grade.

In [15]:
#fillnas with the recurso 
exam_pivot['Normal'].fillna(exam_pivot['Recurso'], inplace = True)

#drop recurso table and rename column to describe what it refers to - an exam_mark
exam_pivot = exam_pivot.drop('Recurso', axis = 1).rename(columns = {'Normal': 'exam_mark'})

In [16]:
#now we remove useless columns and rename nota_final to final_mark
exams = exams.drop(['statusAvaliacao', 'notaAvaliacao'], axis = 1).rename(columns = {'notaFinal' : 'final_mark'})
exams = pd.merge(exams, exam_pivot, on = ['courseid', 'semestre', 'userid'], how = 'inner')

#we finish by dropping the rows that have no final_mark
exams = exams.drop_duplicates().fillna(0).reset_index(drop = True)
exams['courseid'], exams['userid'] = exams['courseid'].astype(object), exams['userid'].astype(object)

exams.describe(include = 'all', datetime_is_numeric = True)

,cd_curso,nm_curso_pt,courseid,ds_discip_pt,semestre,userid,final_mark,exam_mark
count,15289.000000,15289,15289.0,15289,15289,15289.0,15289.000000,15289.000000
unique,NaN,28,251.0,239,6,2305.0,NaN,NaN
top,NaN,Mestrado em Gestão de Informação,200086.0,Business Process Management,S1,3248.0,NaN,NaN
freq,NaN,3338,339.0,356,6216,17.0,NaN,NaN
mean,7284.332330,NaN,NaN,NaN,NaN,NaN,14.911178,14.544688
std,2158.531359,NaN,NaN,NaN,NaN,NaN,3.752871,4.075090
min,859.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,4973.000000,NaN,NaN,NaN,NaN,NaN,14.000000,13.000000
50%,8183.000000,NaN,NaN,NaN,NaN,NaN,16.000000,16.000000
75%,9155.000000,NaN,NaN,NaN,NaN,NaN,17.000000,17.000000


**We now have a table with both of our potential targets**.

Now, we need to go back to our main support table. In this table, we now deal with the remaining assignments - that is, the ones that are identified as N.

One of the first issues is that we have no means to identify each assignment. We can start by giving every semester-course-status a unique identifier

In [17]:
#first, split exams from remaining rows
assignments = support_table['logs_Moodle_cursos'][~(support_table['logs_Moodle_cursos']['statusAvaliacao'].isin(exames))].filter(['cd_curso',
                                                                                                                                 'courseid',                                                                                                                                 'semestre','userid',
                                                                                                                                  'statusAvaliacao', 'cd_final','notaAvaliacao'])
#then we get all unique entries and assign them an index number
assign_id = assignments[['cd_curso', 'semestre', 'courseid', 'statusAvaliacao']].drop_duplicates().reset_index(drop = True).reset_index()

#then, we create a dict using the combination of index, courseid and status as keys
assign_id = assign_id.set_index(['cd_curso', 'semestre', 'courseid', 'statusAvaliacao']).to_dict()['index']

#set index of df to match same index
assignments.set_index(['cd_curso', 'semestre', 'courseid', 'statusAvaliacao'], drop = True, inplace = True)

#use index as key for dict
assignments['assign_id'] = assignments.index.map(assign_id)

#reset_index
assignments = assignments.reset_index().drop(['cd_final'], axis = 1)
assignments['assign_id'], assignments['courseid'] = assignments['assign_id'].astype(object), assignments['courseid'].astype(object)

#before finishing, we still need to make sure we only consider students for which there are, at least, examgrades
assignments = pd.merge(assignments, exams[['cd_curso', 'nm_curso_pt', 'courseid', 'ds_discip_pt', 'semestre', 'userid']], on = ['cd_curso','courseid', 'semestre', 'userid'], how = 'right')

#convert both to objects
assignments

,cd_curso,semestre,courseid,statusAvaliacao,userid,notaAvaliacao,assign_id,nm_curso_pt,ds_discip_pt
0,8259,S1,100001,NaN,1544,NaN,NaN,Licenciatura em Sistemas e Tecnologias de Info...,Álgebra Linear
1,8259,S1,100001.0,Nota Parcial 1,1556.0,13.9,0,Licenciatura em Sistemas e Tecnologias de Info...,Álgebra Linear
2,8259,S1,100001.0,Nota Parcial 2,1556.0,8.7,1,Licenciatura em Sistemas e Tecnologias de Info...,Álgebra Linear
3,9155,S1,100001.0,Nota Parcial 1,1564.0,16.8,2,Licenciatura em Gestão de Informação,Álgebra Linear
4,9155,S1,100001.0,Nota Parcial 2,1564.0,7.3,3,Licenciatura em Gestão de Informação,Álgebra Linear
...,...,...,...,...,...,...,...,...,...
40445,4969,T4,400094.0,Nota Parcial 5,7248.0,12.7,1789,Pós-Graduação em Digital Enterprise Management,Big Data Analytics for Business
40446,4969,T4,400094.0,Nota Parcial 2,7250.0,17.0,1786,Pós-Graduação em Digital Enterprise Management,Big Data Analytics for Business
40447,4969,T4,400094.0,Nota Parcial 3,7250.0,18.0,1787,Pós-Graduação em Digital Enterprise Management,Big Data Analytics for Business
40448,4969,T4,400094.0,Nota Parcial 4,7250.0,17.0,1788,Pós-Graduação em Digital Enterprise Management,Big Data Analytics for Business


In [18]:
assignments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40450 entries, 0 to 40449
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cd_curso         40450 non-null  int64  
 1   semestre         40450 non-null  object 
 2   courseid         40450 non-null  object 
 3   statusAvaliacao  37611 non-null  object 
 4   userid           40450 non-null  object 
 5   notaAvaliacao    37611 non-null  float64
 6   assign_id        37611 non-null  object 
 7   nm_curso_pt      40450 non-null  object 
 8   ds_discip_pt     40450 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.1+ MB


#### Now what?

At the start of this notebook, we aimed at a couple of things:

1. Student Performance -> we will either use exam_mark or the final_mark for this
2. The duration of the course -> for this we need the start date and the end date of the course.

**As a final step, we will store the start date of each course to the support table.**

In [19]:
#perform inner join between support table and courses with grades
support_table = pd.merge(assignments, support_table['datasExames'], on = ['semestre','courseid'], how = 'left')

**We will only touch the course Start Date in the next notebook.**

For now, we will make the following concessions concerning end-date:

1. For every Semester class: S1, S2, T1, T2, T3 and T4, we will find the mean date (which corresponds to the mean of the dates of the normal exam).

2. All courses in each semester class will have an end-date that is equal to the Friday of the week in question.

This will ensure that all courses in the class have the same duration - which will ease our work later.

In [20]:
#then, we cumulative sum all in-group members 
support_table['end_date'] = pd.to_datetime((support_table.groupby('semestre')['dataExame'].transform(pd.Series.mean)).dt.date)

#setting up duration threshold to be on friday -> weekday 4
support_table['end_date'] = support_table['end_date'].where( support_table['end_date'] == (( support_table['end_date'] + Week(weekday=4)) - Week()), support_table['end_date'] + Week(weekday=4))

#this also allows us to remove the pesky columns that serve no additional purpose
support_table = support_table.drop(['epocaAvaliacao', 'dataExame'], axis = 1).drop_duplicates().reset_index(drop = True)

The original end dates, this way, are:

S1: 2021-01-15    
S2: 2021-06-18    
T2: 2021-01-22  
T1: 2020-10-30    
T4: 2021-06-11  
T3: 2021-04-09

we will add a week to S1, S2, and T1 - for semesters it ensures that we cover more days for the 100% duration prediction.
The T1 will ensure the course duration is consistent with the duration of other trimester. 

T2 is to large to consider (it includes christmas break).

In [21]:
end_date = {
    'S1': pd.to_datetime('2021-01-22'),    
    'S2': pd.to_datetime('2021-06-25'),    
    'T2': pd.to_datetime('2021-01-22'),  
    'T1': pd.to_datetime('2020-11-06'),    
    'T4': pd.to_datetime('2021-06-11'),  
    'T3': pd.to_datetime('2021-04-09'),
    }

In [22]:
support_table['end_date'].value_counts()

2021-01-15    16882
2021-06-18    16257
2021-01-22     2104
2020-10-30     1838
2021-06-11     1805
2021-04-09     1564
Name: end_date, dtype: int64

We have an end-date that is suitable - calculated as the mean date of the Normal season exams of the trimester-semester the course is part of.
However, we do not have, yet, a start date.

The logs have start and end dates that are, at the very least, unreliable. A way to address the issue is to look at the curricular unit start dates of MDSAA (which I attended) and extrapolate those start weeks.

As such, we will pre-emptively give courses the following Start Dates:

T1 and S1: Week from 7 of September to 13th of September 2020

T2: 2nd of November 2020

S2 and T3: Week February 8th 2021

T4: 12th of April

**As such, we will go forward with creating a suitable dict to reflect the expected start dates.**

In [23]:
# #Step 1, create dict
startdates = {
            'T1' : pd.to_datetime('2020-09-07'),
            'S1' : pd.to_datetime('2020-09-07'),
            'T2' : pd.to_datetime('2020-11-02'),
            'T3' : pd.to_datetime('2021-02-08'),
            'S2' : pd.to_datetime('2021-02-08'),
            'T4' : pd.to_datetime('2021-04-12'),
            }
                                  
#step 2: assign start date to course
support_table['startdate'] = support_table['semestre'].map(startdates)
support_table['end_date'] = support_table['semestre'].map(end_date)
support_table.rename(columns = {'notaAvaliacao' : 'assignment_mark'}, inplace = True)

In [24]:
support_table.describe(include = 'all')

,cd_curso,semestre,courseid,statusAvaliacao,userid,assignment_mark,assign_id,nm_curso_pt,ds_discip_pt,end_date,startdate
count,40450.000000,40450,40450.0,37611,40450.0,37611.000000,37611.0,40450,40450,40450,40450
unique,NaN,6,251.0,13,2305.0,NaN,1781.0,28,239,5,4
top,NaN,S1,200071.0,Nota Parcial 1,3248.0,NaN,1553.0,Mestrado em Gestão de Informação,Gestão do Conhecimento,2021-01-22 00:00:00,2020-09-07 00:00:00
freq,NaN,16882,1465.0,11489,56.0,NaN,179.0,8175,1465,18986,18720
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-06 00:00:00,2020-09-07 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-25 00:00:00,2021-04-12 00:00:00
mean,7566.668232,NaN,NaN,NaN,NaN,14.511157,NaN,NaN,NaN,NaN,NaN
std,1943.469326,NaN,NaN,NaN,NaN,4.548028,NaN,NaN,NaN,NaN,NaN
min,859.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
25%,5381.000000,NaN,NaN,NaN,NaN,12.700000,NaN,NaN,NaN,NaN,NaN


In [25]:
exams.describe(include = 'all')

,cd_curso,nm_curso_pt,courseid,ds_discip_pt,semestre,userid,final_mark,exam_mark
count,15289.000000,15289,15289.0,15289,15289,15289.0,15289.000000,15289.000000
unique,NaN,28,251.0,239,6,2305.0,NaN,NaN
top,NaN,Mestrado em Gestão de Informação,200086.0,Business Process Management,S1,3248.0,NaN,NaN
freq,NaN,3338,339.0,356,6216,17.0,NaN,NaN
mean,7284.332330,NaN,NaN,NaN,NaN,NaN,14.911178,14.544688
std,2158.531359,NaN,NaN,NaN,NaN,NaN,3.752871,4.075090
min,859.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,4973.000000,NaN,NaN,NaN,NaN,NaN,14.000000,13.000000
50%,8183.000000,NaN,NaN,NaN,NaN,NaN,16.000000,16.000000
75%,9155.000000,NaN,NaN,NaN,NaN,NaN,17.000000,17.000000


We now have 2 distinct table that will be invaluable for our work in future notebooks.

targets_table has stored every final mark obtain by each student attending the different courses of the university:

- From final_mark or exam_mar, we finally are able to calculate our target variables:
    - We can label students as at-risk or as overachievers depending on their mark,


- From support_table, we will need more robust sets of information to be used for feature extraction and engineering:
    - The endate and the expected startdate of each course,
    - The individual mark of each assignment - tbd if usable on a later stage
    
    
- Before finishing, we still have to deal with one thing, which is the proper naming of nans surfacing from the right merge.

In [26]:
#save tables 
exams.to_csv('../Data/Modeling Stage/Nova_IMS_targets_table.csv') 

support_table.to_csv('../Data/Nova_IMS_support_table.csv')

#### Done for now

In notebook 2.2. we will rely on the activity logs and our support table to perform the necessary filtering and preprocessing of the data in order to make it compliant with our necessities. 